In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install biosppy peakutils wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
!pip install pandas==2.2.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wfdb 4.3.0 requires pandas>=2.2.3, but you have pandas 2.2.2 which is incompatible.


In [ ]:
!git clone https://github.com/XianhuiChen/Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network.git
%cd Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network


Cloning into 'Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 184 (delta 32), reused 31 (delta 31), pack-reused 148 (from 1)
Receiving objects: 100% (184/184), 1.28 MiB | 3.81 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network


In [ ]:
from zipfile import ZipFile

uploaded_zip_path = '/content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0.zip'  # <-- update if needed
extract_to_path = '/content/drive/MyDrive/nndl'

with ZipFile(uploaded_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)


In [ ]:
!unzip "/content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0.zip" -d /content/drive/MyDrive/nndl


Archive:  /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0.zip
replace /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/x35.xws? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/x35.xws  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/b04.qrs  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/x24.qrs  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/c06.hea  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/a05.xws  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/c07.apn  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/a02er.hea  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/a18.apn  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/a08.xws  
 extracting: /content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/additional-information.txt  
 extracting: /content/drive/MyDrive/nndl/a

In [ ]:
!sed -i 's/from sklearn.utils import cpu_count/from multiprocessing import cpu_count/' /content/Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network/Preprocessing.py


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from multiprocessing import cpu_count

from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
base_dir = "/content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0"

fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35 if cpu_count() > 35 else cpu_count() - 1  # Setting according to the number of CPU cores


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()

    answers = {}
    with open(os.path.join(base_dir, "/content/Toward-Sleep-Apnea-Detection-with-Lightweight-Multi-scaled-Fusion-Network/dataset/event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
c10: 100%|██████████| 431/431 [01:13<00:00,  5.84it/s]

Testing...
x35: 100%|██████████| 483/483 [01:17<00:00,  6.24it/s]

ok!


In [ ]:
"""NOTES: Batch data is different each time in keras, which result in slight differences in results."""
import pickle
import keras
import numpy as np
import pandas as pd
import os
from keras.layers import Dropout, MaxPooling1D, Reshape, multiply, Conv1D, GlobalAveragePooling1D, Dense
from keras import Input, Model
from keras.models import load_model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from scipy.interpolate import splev, splrep
from sklearn.metrics import confusion_matrix, f1_score
import random

base_dir = "./dataset"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
ir = 3  # interpolate interval
before = 2
after = 2
# normalize
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

def load_data(path):
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))
    with open(os.path.join(base_dir, path), 'rb') as f:  # read preprocessing result
        apnea_ecg = pickle.load(f)
    x_train1,x_train2,x_train3 = [],[],[]
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]
    groups_train = apnea_ecg["groups_train"]
    for i in range(len(o_train)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_train[i]
        # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_train1.append([rri_interp_signal, ampl_interp_signal])  # 5-minute-long segment
        x_train2.append([rri_interp_signal[180:720], ampl_interp_signal[180:720]])  # 3-minute-long segment
        x_train3.append([rri_interp_signal[360:540], ampl_interp_signal[360:540]])  # 1-minute-long segment
    x_training1,x_training2,x_training3,y_training,groups_training = [],[],[],[],[]
    x_val1,x_val2,x_val3,y_val,groups_val = [],[],[],[],[]

    trainlist = random.sample(range(len(o_train)),int(len(o_train)*0.7))
    num = [i for i in range(16709)]
    vallist = set(num) - set(trainlist)
    vallist = list(vallist)
    for i in trainlist:
        x_training1.append(x_train1[i])
        x_training2.append(x_train2[i])
        x_training3.append(x_train3[i])
        y_training.append(y_train[i])
        groups_training.append(groups_train[i])
    for i in vallist:
        x_val1.append(x_train1[i])
        x_val2.append(x_train2[i])
        x_val3.append(x_train3[i])
        y_val.append(y_train[i])
        groups_val.append(groups_train[i])
    x_training1 = np.array(x_training1, dtype="float32").transpose((0, 2, 1))
    x_training2 = np.array(x_training2, dtype="float32").transpose((0, 2, 1))
    x_training3 = np.array(x_training3, dtype="float32").transpose((0, 2, 1))
    y_training = np.array(y_training, dtype="float32")
    x_val1 = np.array(x_val1, dtype="float32").transpose((0, 2, 1))
    x_val2 = np.array(x_val2, dtype="float32").transpose((0, 2, 1))
    x_val3 = np.array(x_val3, dtype="float32").transpose((0, 2, 1))
    y_val = np.array(y_val, dtype="float32")
    x_test1,x_test2,x_test3 = [],[],[]
    o_test, y_test = apnea_ecg["o_test"], apnea_ecg["y_test"]
    groups_test = apnea_ecg["groups_test"]
    for i in range(len(o_test)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_test[i]
        # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_test1.append([rri_interp_signal, ampl_interp_signal])
        x_test2.append([rri_interp_signal[180:720], ampl_interp_signal[180:720]])
        x_test3.append([rri_interp_signal[360:540], ampl_interp_signal[360:540]])
    x_test1 = np.array(x_test1, dtype="float32").transpose((0, 2, 1))
    x_test2 = np.array(x_test2, dtype="float32").transpose((0, 2, 1))
    x_test3 = np.array(x_test3, dtype="float32").transpose((0, 2, 1))
    y_test = np.array(y_test, dtype="float32")

    return x_training1, x_training2, x_training3, y_training, groups_training, x_val1, x_val2, \
           x_val3, y_val, groups_val, x_test1, x_test2, x_test3, y_test, groups_test


def lr_schedule(epoch, lr):
    if epoch > 70 and (epoch - 1) % 10 == 0:
        lr *= 0.1
    print("Learning rate: ", lr)
    return lr


def create_model(input_a_shape, input_b_shape, input_c_shape, weight=1e-3):
    # SA-CNN-3
    input1 = Input(shape=input_a_shape)
    x1 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight))(input1)
    x1 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x1)
    x1 = MaxPooling1D(pool_size=3, padding="same")(x1)
    x1 = Conv1D(32, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x1)
    x1 = MaxPooling1D(pool_size=5, padding="same")(x1)

    # SA-CNN-2
    input2 = Input(shape=input_b_shape)
    x2 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight))(input2)
    x2 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x2)
    x2 = MaxPooling1D(pool_size=3, padding="same")(x2)
    x2 = Conv1D(32, kernel_size=11, strides=3, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x2)

    # SA-CNN-1
    input3 = Input(shape=input_c_shape)
    x3 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight))(input3)
    x3 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x3)
    x3 = MaxPooling1D(pool_size=3, padding="same")(x3)
    x3 = Conv1D(32, kernel_size=1, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(1e-3), bias_regularizer=l2(weight))(x3)

    # Channel-wise attention module
    concat = keras.layers.concatenate([x1, x2, x3], name="Concat_Layer", axis=-1)
    squeeze = GlobalAveragePooling1D()(concat)
    excitation = Dense(48, activation='relu')(squeeze)
    excitation = Dense(96, activation='sigmoid')(excitation)
    excitation = Reshape((1, 96))(excitation)
    scale = multiply([concat, excitation])
    x = GlobalAveragePooling1D()(scale)
    dp = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax', name="Output_Layer")(dp)
    model = Model(inputs=[input1, input2, input3], outputs=outputs)
    return model


if __name__ == "__main__":
    # load_data
    path = "/content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/apnea-ecg.pkl"
    x_train1, x_train2, x_train3, y_train, groups_train, x_val1, x_val2,\
    x_val3, y_val, groups_val, x_test1, x_test2, x_test3, y_test, groups_test = load_data(path)
    y_train = keras.utils.to_categorical(y_train, num_classes=2)  # Convert to two categories
    y_val = keras.utils.to_categorical(y_val, num_classes=2)
    y_test = keras.utils.to_categorical(y_test, num_classes=2)
    print('input_shape', x_train1.shape, x_train2.shape, x_train3.shape)

    # training
    model = create_model(x_train1.shape[1:], x_train2.shape[1:], x_train3.shape[1:])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "/content/drive/MyDrive/nndl/weights.best (1)/weights.best.keras"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callbacks_list = [lr_scheduler, checkpoint]
    history = model.fit([x_train1, x_train2, x_train3], y_train, batch_size=128, epochs=1,
                        validation_data=([x_val1, x_val2, x_val3], y_val), callbacks=callbacks_list)

    # test
    filepath = '/content/drive/MyDrive/nndl/weights.best (1)/weights.best.hdf5'
    model = load_model(filepath)
    loss, accuracy = model.evaluate([x_test1, x_test2, x_test3], y_test)
    # save prediction score
    y_score = model.predict([x_test1, x_test2, x_test3])
    output = pd.DataFrame({"y_true": y_test[:, 1], "y_score": y_score[:, 1], "subject": groups_test})
    output.to_csv("/content/drive/MyDrive/nndl/SE-MSCNN.csv", index=False)
    y_true, y_pred = np.argmax(y_test, axis=-1), np.argmax(model.predict([x_test1, x_test2, x_test3], batch_size=1024, verbose=1), axis=-1)
    C = confusion_matrix(y_true, y_pred, labels=(1, 0))
    TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
    acc, sn, sp = 1. * (TP + TN) / (TP + TN + FP + FN), 1. * TP / (TP + FN), 1. * TN / (TN + FP)
    f1 = f1_score(y_true, y_pred, average='binary')
    print("acc: {}, sn: {}, sp: {}, f1: {}".format(acc, sn, sp, f1))



input_shape (11696, 900, 2) (11696, 540, 2) (11696, 180, 2)
Learning rate:  0.0010000000474974513
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.5857 - loss: 1.0322
Epoch 1: val_accuracy improved from -inf to 0.57909, saving model to /content/drive/MyDrive/nndl/weights.best (1)/weights.best.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 32s 291ms/step - accuracy: 0.5859 - loss: 1.0314 - val_accuracy: 0.5791 - val_loss: 0.8304 - learning_rate: 0.0010


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


530/530 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.9040 - loss: 0.3178
530/530 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 11s 640ms/step
acc: 0.9064620832103866, sn: 0.8600924499229584, sp: 0.935246293639407, f1: 0.8756765236489137


In [ ]:
import pickle
import numpy as np
import pandas as pd
import os
from scipy.interpolate import splev, splrep
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

base_dir = "./dataset"
ir = 3
before = 2
after = 2

# normalize
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

def load_flattened_data(path):
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))
    with open(os.path.join(base_dir, path), 'rb') as f:
        apnea_ecg = pickle.load(f)

    def interpolate_features(o_data):
        features = []
        for (rri_tm, rri_signal), (ampl_tm, ampl_signal) in o_data:
            rri_interp = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
            ampl_interp = splev(tm, splrep(ampl_tm, scaler(ampl_signal), k=3), ext=1)
            # Combine into one flat feature vector
            combined = np.hstack((rri_interp, ampl_interp))
            features.append(combined)
        return np.array(features)

    X_train = interpolate_features(apnea_ecg["o_train"])
    y_train = np.array(apnea_ecg["y_train"])
    X_test = interpolate_features(apnea_ecg["o_test"])
    y_test = np.array(apnea_ecg["y_test"])
    groups_test = apnea_ecg["groups_test"]

    return X_train, y_train, X_test, y_test, groups_test

# Load data
path = "/content/drive/MyDrive/nndl/apnea-ecg-database-1.0.0/apnea-ecg.pkl"
X_train, y_train, X_test, y_test, groups_test = load_flattened_data(path)

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# --- SVM Classifier ---
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_probs = svm_model.predict_proba(X_test)[:, 1]

# --- KNN Classifier ---
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
knn_preds = knn_model.predict(X_test)
knn_probs = knn_model.predict_proba(X_test)[:, 1]

def evaluate_model(name, y_true, y_pred, y_probs, groups_test, save_path):
    C = confusion_matrix(y_true, y_pred, labels=(1, 0))
    TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
    acc = (TP + TN) / (TP + TN + FP + FN)
    sn = TP / (TP + FN)
    sp = TN / (TN + FP)
    f1 = f1_score(y_true, y_pred, average='binary')

    print(f"\n{name} Results:")
    print(f"Accuracy: {acc:.4f}, Sensitivity: {sn:.4f}, Specificity: {sp:.4f}, F1 Score: {f1:.4f}")

    # Save CSV with scores
    pd.DataFrame({
        "y_true": y_true,
        "y_score": y_probs,
        "subject": groups_test
    }).to_csv(save_path, index=False)

# Evaluate and Save
evaluate_model("SVM", y_test, svm_preds, svm_probs, groups_test, "/content/drive/MyDrive/nndl/SVM.csv")
evaluate_model("KNN", y_test, knn_preds, knn_probs, groups_test, "/content/drive/MyDrive/nndl/KNN.csv")



SVM Results:
Accuracy: 0.7833, Sensitivity: 0.6145, Specificity: 0.8881, F1 Score: 0.6848

KNN Results:
Accuracy: 0.7432, Sensitivity: 0.5414, Specificity: 0.8685, F1 Score: 0.6176


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score

def evaluate_method(csv_path, model_name):
    # Load test output
    df = pd.read_csv(csv_path)
    df["y_pred"] = df["y_score"] > 0.5

    # Group by subject, calculate apnea frequency per hour
    df = df.groupby("subject", group_keys=False).apply(lambda d: d["y_pred"].mean() * 60)
    df.name = model_name
    return df

# Load predictions from each model
semscnn = evaluate_method("/content/drive/MyDrive/nndl/SE-MSCNN.csv", "SEMSCNN")
svm = evaluate_method("/content/drive/MyDrive/nndl/SVM.csv", "SVM")
knn = evaluate_method("/content/drive/MyDrive/nndl/KNN.csv", "KNN")

# Load original annotation summary
original = []
with open("/content/drive/MyDrive/nndl/additional-information.txt", "r") as f:
    for line in f:
        rows = line.strip().split("\t")
        if len(rows) == 12 and rows[0].startswith("x"):
            original.append([rows[0], float(rows[3]) / float(rows[1]) * 60])
original = pd.DataFrame(original, columns=["subject", "original"]).set_index("subject")

# Combine all predictions
all_df = pd.concat([semscnn, svm, knn, original], axis=1)

# Evaluation function
def evaluate_model(name):
    model_scores = all_df[[name, "original"]].dropna()
    corr_value = model_scores.corr().iloc[0, 1]
    binarized = model_scores.applymap(lambda x: int(x > 5))
    C = confusion_matrix(binarized["original"], binarized[name], labels=(1, 0))
    TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
    acc = (TP + TN) / (TP + TN + FP + FN)
    sn = TP / (TP + FN)
    sp = TN / (TN + FP)
    auc = roc_auc_score(model_scores["original"] > 5, model_scores[name])
    print(f"\n{name} Results:")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Sensitivity: {sn * 100:.2f}%")
    print(f"Specificity: {sp * 100:.2f}%")
    print(f"AUC: {auc:.3f}")
    print(f"Correlation: {corr_value:.3f}")

# Evaluate each model
evaluate_model("SEMSCNN")
evaluate_model("SVM")
evaluate_model("KNN")



SEMSCNN Results:
Accuracy: 100.00%
Sensitivity: 100.00%
Specificity: 100.00%
AUC: 1.000
Correlation: 0.979

SVM Results:
Accuracy: 88.57%
Sensitivity: 95.65%
Specificity: 75.00%
AUC: 0.928
Correlation: 0.765

KNN Results:
Accuracy: 85.71%
Sensitivity: 100.00%
Specificity: 58.33%
AUC: 0.978
Correlation: 0.849


<ipython-input-7-5425d0258311>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("subject", group_keys=False).apply(lambda d: d["y_pred"].mean() * 60)
<ipython-input-7-5425d0258311>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("subject", group_keys=False).apply(lambda d: d["y_pred"].mean() * 60)
<ipython-input-7-5425d0258311>:11: DeprecationWarning: DataFrameGroupBy.appl